# Harmony API Introduction

This notebook provides an overview of the capabilities offered through the Harmony API and SWOT L2 Reproject tool. While written for SWOT L2 data, it works with most any level 2 data for projecting to a normal grid. In this tutorial we will use MODIS L2 data to show the native file projected to equal-area-cylindracal projection using both Nearest Neighbor and Bi-linear interpolation.

Standing on the shoulders of previous authors: Amy Steiker, Patrick Quinn

## Import packages

Most packages below should be included natively with the Anaconda Python distribution, for example, but some may need to install packages like `rasterio` manually using the following example:

In [ ]:
# Install prerequisite packages
import sys
!{sys.executable} -m pip install rasterio # Install a pip package in the current Jupyter kernel
!{sys.executable} -m pip install requests # Install a pip package in the current Jupyter kernel

In [ ]:
from urllib import request, parse
from http.cookiejar import CookieJar
import getpass
import netrc
import os
import requests
import json
import pprint
from osgeo import gdal
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import rasterio
from rasterio.plot import show
import numpy as np
import os
import time
from netCDF4 import Dataset
%matplotlib inline

## Earthdata Login Authentication

An Earthdata Login account is required to access data from NASA EOSDIS. In order to access data from the Harmony API, you will need to create an account in the Earthdata Login UAT environment. Please visit https://uat.urs.earthdata.nasa.gov to set up an account in this test environment. These accounts, as all Earthdata Login accounts, are free to create and only take a moment to set up.



We need some boilerplate up front to log in to Earthdata Login.  The function below will allow Python scripts to log into any Earthdata Login application programmatically.  To avoid being prompted for
credentials every time you run and also allow clients such as curl to log in, you can add the following
to a `.netrc` (`_netrc` on Windows) file in your home directory:

```
machine uat.urs.earthdata.nasa.gov
    login <your username>
    password <your password>
```

Make sure that this file is only readable by the current user or you will receive an error stating
"netrc access too permissive."

`$ chmod 0600 ~/.netrc` 


In [ ]:
def setup_earthdata_login_auth(endpoint):
    """
    Set up the request library so that it authenticates against the given Earthdata Login
    endpoint and is able to track cookies between requests.  This looks in the .netrc file 
    first and if no credentials are found, it prompts for them.

    Valid endpoints include:
        uat.urs.earthdata.nasa.gov - Earthdata Login UAT (Harmony's current default)
        urs.earthdata.nasa.gov - Earthdata Login production
    """
    try:
        username, _, password = netrc.netrc().authenticators(endpoint)
    except (FileNotFoundError, TypeError):
        # FileNotFound = There's no .netrc file
        # TypeError = The endpoint isn't in the netrc file, causing the above to try unpacking None
        print('Please provide your Earthdata Login credentials to allow data access')
        print('Your credentials will only be passed to %s and will not be exposed in Jupyter' % (endpoint))
        username = input('Username:')
        password = getpass.getpass()

    manager = request.HTTPPasswordMgrWithDefaultRealm()
    manager.add_password(None, endpoint, username, password)
    auth = request.HTTPBasicAuthHandler(manager)

    jar = CookieJar()
    processor = request.HTTPCookieProcessor(jar)
    opener = request.build_opener(auth, processor)
    request.install_opener(opener)

Now call the above function to set up Earthdata Login for subsequent requests

In [ ]:
setup_earthdata_login_auth('uat.urs.earthdata.nasa.gov')

## Identify a data collection of interest

A CMR collection ID is needed to request services through Harmony. The collection ID can be determined using the [CMR API](https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html). We will query the corresponding ID of a known collection short name, `harmony_example`, which is a Level 3 test collection with transformation services available through Harmony.

In [ ]:
params = {
    'short_name': 'MODIS_A-JPL-L2P-v2019.0',
    'provider_id': 'POCLOUD'
} # parameter dictionary with known CMR short_name

cmr_collections_url = 'https://cmr.uat.earthdata.nasa.gov/search/collections.json'
cmr_response = requests.get(cmr_collections_url, params=params)
cmr_results = json.loads(cmr_response.content) # Get json response from CMR collection metadata

collectionlist = [el['id'] for el in cmr_results['feed']['entry']]
harmony_collection_id = collectionlist[0]
print(harmony_collection_id)

We can also view the `MODIS_A-JPL-L2P-v2019.0` collection metadata to glean more information about the collection:

In [ ]:
pprint.pprint(cmr_results)

## Access reprojected data

The Harmony API accepts reprojection requests with a given coordinate reference system using the `outputCrs` keyword. According to the Harmony API documentation, this keyword "recognizes CRS types that can be inferred by gdal, including EPSG codes, Proj4 strings, and OGC URLs (http://www.opengis.net/def/crs/...) ". Two examples below demonstrate inputting an EPSG code and Proj4 string using the global test granule from previous examples. First, let's view the projection information of the granule in the native projection, using the variable subset example:

## Access Level 2 swath regridded data

Moving outside of the `harmony/gdal` service, we will now request regridding from the `sds/swot-reproject` service using the `C1234724470-POCLOUD`, or Harmony L2 swath example, collection provided in NetCDF format. 


The Harmony API accepts several query parameters related to regridding and interpolation in addition to the reprojection parameters above: 

`interpolation=<String>` - Both `near` and `bilinear` are valid options

`scaleSize=x,y` - 2 comma separated numbers as floats

`scaleExtent=xmin,ymin,xmax,ymax` - 4 comma separated numbers as floats

`width=<Float>`  

`height=<Float>` 

An error is returned if both `scaleSize` and `width`/`height` parameters are both provided (only one or the other can be used).

Request reprojection to [Europe Lambert Conformal Conic](https://epsg.io/102014) with a new scale extent and nearest neighbor interpolation:

In [ ]:
harmony_root = 'https://harmony.uat.earthdata.nasa.gov'

# URL encode string using urllib parse package
proj_string = '+proj=cea +lon_0=0 +lat_ts=30 +x_0=0 +y_0=0 +ellps=WGS84 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs' # proj4 of WGS 84 / NSIDC EASE-Grid 2.0 Global projection
#l2proj_string = '+proj=laea +lat_0=52 +lon_0=10 +x_0=4321000 +y_0=3210000 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs'
l2proj_encode = parse.quote(proj_string)

harmony_root = 'https://harmony.uat.earthdata.nasa.gov'

regridConfig = {
    'l2collection_id': 'C1234724470-POCLOUD',
    'ogc-api-coverages_version': '1.0.0',
    'variable': 'all',
    'granuleid': 'G1234734747-POCLOUD',
    'outputCrs': l2proj_encode,
    'interpolation': 'near',
    'width': 1000,
    'height': 1000
}

regrid_url = harmony_root+'/{l2collection_id}/ogc-api-coverages/{ogc-api-coverages_version}/collections/{variable}/coverage/rangeset?&granuleid={granuleid}&outputCrs={outputCrs}&interpolation={interpolation}&height={height}&width={width}'.format(**regridConfig)
print('Request URL', regrid_url)
regrid_response = request.urlopen(regrid_url)
regrid_results = regrid_response.read()

This reprojected and regridded output is downloaded to the Harmony outputs directory and we can inspect a variable to check for projection and grid dimension:

In [ ]:
regrid_file_name = 'regrid-near.nc'
regrid_filepath = str(regrid_file_name)
file_ = open(regrid_filepath, 'wb')
file_.write(regrid_results)
file_.close()

In [ ]:
harmony_root = 'https://harmony.uat.earthdata.nasa.gov'

# URL encode string using urllib parse package
proj_string = '+proj=cea +lon_0=0 +lat_ts=30 +x_0=0 +y_0=0 +ellps=WGS84 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs' # proj4 of WGS 84 / NSIDC EASE-Grid 2.0 Global projection
#l2proj_string = '+proj=laea +lat_0=52 +lon_0=10 +x_0=4321000 +y_0=3210000 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs'
l2proj_encode = parse.quote(proj_string)

harmony_root = 'https://harmony.uat.earthdata.nasa.gov'

regridConfig = {
    'l2collection_id': 'C1234724470-POCLOUD',
    'ogc-api-coverages_version': '1.0.0',
    'variable': 'all',
    'granuleid': 'G1234734747-POCLOUD',
    'outputCrs': l2proj_encode,
    'interpolation': 'bilinear',
    'width': 1000,
    'height': 1000
}

regrid_bi_url = harmony_root+'/{l2collection_id}/ogc-api-coverages/{ogc-api-coverages_version}/collections/{variable}/coverage/rangeset?&granuleid={granuleid}&outputCrs={outputCrs}&interpolation={interpolation}&height={height}&width={width}'.format(**regridConfig)
print('Request URL', regrid_bi_url)
regrid_bi_response = request.urlopen(regrid_bi_url)
regrid_bi_results = regrid_bi_response.read()

In [ ]:
regrid_bi_file_name = 'regrid-bi.nc'
regrid_bi_filepath = str(regrid_bi_file_name)
file_ = open(regrid_bi_filepath, 'wb')
file_.write(regrid_bi_results)
file_.close()

Print the x and y dimensions to confirm that the output matches the requested scale extent in meters:

In [ ]:
import xarray as xa
reproject_ds = xa.open_dataset(regrid_filepath, drop_variables='time')
print(reproject_ds)

In [ ]:
import xarray as xa
reproject_bi_ds = xa.open_dataset(regrid_bi_filepath, drop_variables='time')
print(reproject_bi_ds)

In [ ]:
original_ds = xa.open_dataset('20200131234501-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.0.nc')
print(original_ds)

In [ ]:
g = reproject_ds.sea_surface_temperature.plot(robust=True)
g.axes.set_title("Nearest Neighbor Interpolation")

In [ ]:
g= reproject_bi_ds.sea_surface_temperature.plot(robust=True)
g.axes.set_title("Bilinear Interpolation")

In [ ]:
g = original_ds.sea_surface_temperature.plot(robust=True)
g.axes.set_title("Native File")


In [ ]:
g= original_ds.sea_surface_temperature.plot(x="lon", y="lat", robust=True)
g.axes.set_title("Native, projected to Lat/Lon")

In [ ]:

harmony_root = 'https://harmony.uat.earthdata.nasa.gov'

# URL encode string using urllib parse package
proj_string = '+proj=lcc +lon_0=-90 +lat_1=33 +lat_2=45'
l2proj_encode = parse.quote(proj_string)

harmony_root = 'https://harmony.uat.earthdata.nasa.gov'

regridConfig = {
    'l2collection_id': 'C1234724470-POCLOUD',
    'ogc-api-coverages_version': '1.0.0',
    'variable': 'all',
    'granuleid': 'G1234734747-POCLOUD',
    'outputCrs': l2proj_encode,
    'interpolation': 'near',
    'width': 1000,
    'height': 1000
}

regrid_url = harmony_root+'/{l2collection_id}/ogc-api-coverages/{ogc-api-coverages_version}/collections/{variable}/coverage/rangeset?&granuleid={granuleid}&outputCrs={outputCrs}&interpolation={interpolation}&height={height}&width={width}'.format(**regridConfig)
print('Request URL', regrid_url)
regrid_response = request.urlopen(regrid_url)
regrid_results = regrid_response.read()

In [ ]:
regrid_lambert_file_name = 'regrid-lambert.nc'
file_ = open(regrid_lambert_file_name, 'wb')
file_.write(regrid_bi_results)
file_.close()

In [ ]:
import xarray as xa
reproject_lambert_ds = xa.open_dataset(regrid_lambert_file_name, drop_variables='time')
print(reproject_lambert_ds)

In [ ]:
import cartopy.crs as ccrs

f, ax1 = plt.subplots(1, 1, figsize=(8, 6), subplot_kw={'projection': ccrs.LambertConformal()})

g=reproject_lambert_ds.sea_surface_temperature.plot(ax=ax1,  robust=True )
ax1.set_title("Lambert, Nearest Neighbor")
#ax1.set_extent([-160, -150,30, 50])
#ax1.coastlines()
